In [ ]:
import arcpy
import arcgis
import pandas as pd
import numpy as np
import datetime
import requests
import os
import warnings
import matplotlib.pyplot as plt
import rasterio
import os
import psycopg2


In [ ]:
#data source link

# MN temperature data in 03/2023
temperature_url = r"https://mesonet.agron.iastate.edu/api/1/daily.geojson?network=MN_RWIS&year=2023&month=3"

# latest MN Land Cover data in 2019
landcover_path = r"C:\Users\Maochuan\OneDrive\桌面\lab2\NLCD_2019_Land_Cover.tif"

# MN 30m DEM (Raster in GDB from MN Geospatial Commons)
elevation_path = r"C:\Users\Maochuan\OneDrive\桌面\lab2\elev_30m_digital_elevation_model.gdb\digital_elevation_model_30m"

# Output FGDB Path
out_local = r"C:\Users\Maochuan\OneDrive\文档\ArcGIS\Projects\lab2_arc2\lab2_arc2.gdb"

In [ ]:
#request and pull data via url
temperature_response = requests.get(temperature_url)

temperature_json = temperature_response.json()["features"]


#define a fuction to extract data
def ExtractData(data,col,jasonfield):
    data[col] = data[jasonfield].apply(lambda x: dict(x)[col])

#transform geojason as dataframe
temperature_rawdf = pd.DataFrame.from_records(temperature_json)

#property info
temperature_col = ["id", "date", "name", "max_tmpf", "min_tmpf"]

for col_name in temperature_col:
    ExtractData(temperature_rawdf,col_name,"properties")

#location info
temperature_rawdf['x'] = temperature_rawdf["geometry"].apply(lambda x: dict(x)["coordinates"][0])
temperature_rawdf['y'] = temperature_rawdf["geometry"].apply(lambda x: dict(x)["coordinates"][1])

temperature_df = temperature_rawdf[["id", "date", "name", "max_tmpf", "min_tmpf",'x','y']].copy()

#missing values (temperature and location)
temperature_df = temperature_df.dropna(subset=["max_tmpf", "min_tmpf","x", "y"])
temperature_df["date"] = temperature_df["date"].astype('datetime64[ns]')
temperature_df["month"] = pd.DatetimeIndex(temperature_df["date"]).month

#bounding box
temperature_df =temperature_df[(temperature_df["x"] > -97.239) & (temperature_df["x"] < -89.492) &
                              (temperature_df["y"] > 43.499) & (temperature_df["y"] < 49.384)]

#outliers (mean+-3*std by station id AND month)
temperature_df = temperature_df[(temperature_df['min_tmpf']<=temperature_df['max_tmpf'])]

m = (temperature_df.groupby(['id','month'])['min_tmpf'].
     transform(lambda s: s.between(s.mean() - 3 * s.std(), s.mean() + 3 * s.std())))

temperature_df = temperature_df[m]

m = (temperature_df.groupby(['id','month'])['max_tmpf'].
     transform(lambda s: s.between(s.mean() - 3 * s.std(), s.mean() + 3 * s.std())))

temperature_df = temperature_df[m]



In [ ]:
temperature_df


In [ ]:
#read data
raster_data = rasterio.open(landcover_path)

# Plot raster data
plt.imshow(raster_data.read(1), cmap='viridis')
plt.title('Land Cover Raster')
plt.colorbar()
plt.show()

In [ ]:
#bounding box
arcpy.management.Clip(landcover_path, "132660 4774410 791819 5491608", os.path.join(out_local, "lc_final"));




In [ ]:
#check if raster is within bounding box
bounding_box = (132660, 4774410, 791819, 5491608)


raster_file = r"C:\Users\Maochuan\OneDrive\桌面\lab2\NLCD_2019_Land_Cover.tif"
with rasterio.open(raster_file) as src:
    # Get the raster bounding box
    raster_bbox = src.bounds
    
# Check if the raster is within the bounding box
    is_within_bbox = (
        raster_bbox.left >= bounding_box[0] and
        raster_bbox.bottom >= bounding_box[1] and
        raster_bbox.right <= bounding_box[2] and
        raster_bbox.top <= bounding_box[3]
    )

# Print the result
if is_within_bbox:
    print("The raster is within the bounding box.")
else:
    print("The raster is not within the bounding box.")

In [ ]:
# Check if the data is categorical or not

# Set the workspace environment
arcpy.env.workspace = r"C:\Users\Maochuan\OneDrive\文档\ArcGIS\Projects\lab2_arc2"

# Define the land cover raster dataset
land_cover_tif = r"C:\Users\Maochuan\OneDrive\桌面\lab2\NLCD_2019_Land_Cover.tif"

# Conver the raster to numpy array
land_cover_array = arcpy.RasterToNumPyArray(land_cover_tif)

# Calculate the unique values in an array
unique_values = np.unique(land_cover_array)

# Set a threshold for the number of unique values to consider a raster categorical
threshold = 100

if len(unique_values) <= threshold:
    print("The land cover GeoTIFF is likely categorical.")
else:
    print("The land cover GeoTIFF is not categorical.")

# In this approach, we first convert the raster to a NumPy array
# then calculate the number of unique values in the array. 
# If the number of unique values is below a certain threshold (e.g., 100), 
# we consider the raster to be categorical. This is not a definitive method, 
# as the threshold value is arbitrary and could vary depending on your specific dataset. 


In [ ]:
# Check for null values
null_values = arcpy.management.GetRasterProperties(landcover_path, "ANYNODATA").getOutput(0)
if null_values == "1":
        print("Null values exist.")
else:
        print("Null values do not exist.")

In [ ]:
#coordinate system
with raster_data as src:
    raster_crs = src.crs

    print("Raster CRS:", raster_crs)

In [ ]:
#cell size

with raster_data as src1:
    # Get the cell size (resolution) from the transform object
    cell_size_x = src1.transform[0]
    cell_size_y = -src1.transform[4]

# Print the cell size
print(f"Cell size (X): {cell_size_x} units")
print(f"Cell size (Y): {cell_size_y} units")

In [ ]:
#read data

elevation_path = r"C:\Users\Maochuan\OneDrive\桌面\lab2\DEM.tif"


In [ ]:
#visual inspection

#read data
raster_data = rasterio.open(elevation_path)

# Plot raster data
plt.imshow(raster_data.read(1), cmap='viridis')
plt.title('Land Cover Raster')
plt.colorbar()
plt.show()

In [ ]:
#bounding box
#check if raster is within bounding box
bounding_box = (132660, 4774410, 791819, 5491608)


DEM_file = r"C:\Users\Maochuan\OneDrive\桌面\lab2\DEM.tif"
with rasterio.open(DEM_file) as src:
    # Get the raster bounding box
    raster_bbox = src.bounds
    
# Check if the raster is within the bounding box
    is_within_bbox = (
        raster_bbox.left >= bounding_box[0] and
        raster_bbox.bottom >= bounding_box[1] and
        raster_bbox.right <= bounding_box[2] and
        raster_bbox.top <= bounding_box[3]
    )

# Print the result
if is_within_bbox:
    print("The raster is within the bounding box.")
else:
    print("The raster is not within the bounding box.")

In [ ]:
#missing value
null_values = arcpy.management.GetRasterProperties(elevation_path, "ANYNODATA").getOutput(0)
if null_values == "1":
        print("Null values exist.")
else:
        print("Null values do not exist.")

In [ ]:
import rasterio
import numpy as np

def find_outliers_iqr(data):
    q1 = np.percentile(data, 25)
    q3 = np.percentile(data, 75)
    iqr = q3 - q1
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
    return lower_bound, upper_bound


with rasterio.open(DEM_file) as src:
    # Read the first band (assuming a single-band raster)
    band = src.read(1)

    # Flatten the 2D array into 1D and remove NaN values
    band_data = band.flatten()
    band_data = band_data[np.isfinite(band_data)]

    # Find outliers using the IQR method
    lower_bound, upper_bound = find_outliers_iqr(band_data)
    
    print(f"Lower bound: {lower_bound}, Upper bound: {upper_bound}")
    print(f"Number of unique values in raster: {len(np.unique(band_data))}")

    # Create a mask for outliers
    outlier_mask = np.logical_or(band < lower_bound, band > upper_bound)

    # Get the row and column indices of the outliers
    row_indices, col_indices = np.where(outlier_mask)

    # Print the outlier pixel values and their indices
    outliers_found = False
    for row, col in zip(row_indices, col_indices):
        print(f"Outlier pixel value: {band[row, col]}, Row: {row}, Column: {col}")
        outliers_found = True
    
    if not outliers_found:
        print("No outliers found in the raster.")


In [ ]:
#get data and data cleaning
stinkbug_path = r"C:\Users\Maochuan\OneDrive\桌面\lab2\mappings.csv"
stinkbug_df = pd.read_csv(stinkbug_path, encoding = 'unicode_escape')
stinkbug_df = stinkbug_df[["objectid", "ObsDate", "Location", "Latitude", "Longitude", "Verified"]].copy()

#get valid data and remove missing data
stinkbug_df = stinkbug_df[stinkbug_df['Verified']=='Verified']
stinkbug_df = stinkbug_df.dropna(subset=["Latitude", "Longitude"])

# Filter where Location Contains 'Minnesota' and using bounding box
stinkbug_df = stinkbug_df[(stinkbug_df["Longitude"] > -97.239) & (stinkbug_df["Longitude"] < -89.492) &
                              (stinkbug_df["Latitude"] > 43.499) & (stinkbug_df["Latitude"] < 49.384)]
stinkbug_df = stinkbug_df[stinkbug_df["Location"].str.contains("Minnesota")]

# Convert DateTime Data Types and get 2023 data
stinkbug_df["ObsDate"] = stinkbug_df["ObsDate"].astype('datetime64[ns]')
stinkbug_df['ObsMonth'] = pd.DatetimeIndex(stinkbug_df["ObsDate"]).month
stinkbug_df = stinkbug_df[pd.DatetimeIndex(stinkbug_df["ObsDate"]).year==2023]

stinkbug_df

In [ ]:
import os
import subprocess
import sys

# Set the input raster file path and output PostGIS raster table name
input_raster = r"C:\raster\final.tif"
raster_table = "raster_table"

# Set the PostGIS connection parameters
postgis_host = "35.223.186.20"
postgis_port = "5432"
postgis_user = "postgres"
postgis_password = "139571wang"
postgis_db = "postgres"

# Set the SRID for the raster data
srid = "4269"

# Set the PATH environment variable for the subprocess
postgis_bin_path = r"C:\Program Files\PostgreSQL\15\bin"  # Replace <version> with your PostgreSQL version
os.environ["PATH"] = postgis_bin_path + os.pathsep + os.environ["PATH"]

# Set the PGPASSWORD environment variable for the raster2pgsql command
os.environ["PGPASSWORD"] = postgis_password

# Build the raster2pgsql command with the tiling option
raster2pgsql_cmd = f"raster2pgsql -s {srid} -I -C -M -t 128x128 {input_raster} -F {raster_table} | psql -h {postgis_host} -p {postgis_port} -U {postgis_user} -d {postgis_db}"

# Execute the raster2pgsql command
process = subprocess.run(raster2pgsql_cmd, shell=True, stderr=subprocess.PIPE, text=True)
if process.returncode != 0:
    print("An error occurred while running the raster2pgsql command:")
    print(process.stderr)
    sys.exit(1)

print("Raster data successfully imported to PostGIS.")


In [ ]:
postgis_host = "35.223.186.20"
postgis_port = "5432"
postgis_user = "postgres"
postgis_password = "139571wang"
postgis_db = "postgres"
csv_file = r"C:\raster\mappings.csv"

data = pd.read_csv(csv_file)

# Connect to the PostGIS database
conn = psycopg2.connect(host=postgis_host, port=postgis_port, user=postgis_user, password=postgis_password, dbname=postgis_db)
cur = conn.cursor()

table_name = "csv_data"
columns = """
    objectid integer,
    reporter varchar,
    RecOwner varchar,
    SciName varchar,
    ComName varchar,
    Nativity varchar,
    OccStatus varchar,
    Status varchar,
    ObsDate varchar,
    DateEnt varchar,
    DateUp varchar,
    Location varchar,
    Latitude double precision,
    Longitude double precision,
    Datum varchar,
    Method varchar,
    CoordAcc varchar,
    DataType varchar,
    Centroid varchar,
    Abundance varchar,
    InfestAcre varchar,
    GrossAcre varchar,
    Percentcov varchar,
    Density varchar,
    Quantity varchar,
    QuantityU varchar,
    APPXQuant varchar,
    NumCollect varchar,
    Smallest varchar,
    Largest varchar,
    Incidence varchar,
    Severity varchar,
    Host varchar,
    Host_Name varchar,
    HostPheno varchar,
    HostDamage varchar,
    ManageStat varchar,
    PopStat varchar,
    Habitat varchar,
    LocalOwner varchar,
    Site varchar,
    RecBasis varchar,
    Museum varchar,
    MuseumRec varchar,
    Voucher varchar,
    ObsIDer varchar,
    CollectTme varchar,
    UUID varchar,
    OrgSrcID varchar,
    OrigName varchar,
    RecSrcTyp varchar,
    Surveyor varchar,
    DateAcc varchar,
    VisitType varchar,
    DataMthd varchar,
    TrapType varchar,
    NumTraps varchar,
    TargetName varchar,
    TargetCnt varchar,
    TargetRnge varchar,
    Phenology varchar,
    LifeStatus varchar,
    Sex varchar,
    PID varchar,
    WaterName varchar,
    WaterType varchar,
    Substrate varchar,
    TreatArea varchar,
    PlantTreat varchar,
    TreatComm varchar,
    Reference varchar,
    Locality varchar,
    Comments varchar,
    ReviewDate varchar,
    Reviewer varchar,
    VerifyMthd varchar,
    Verified varchar,
    IDCred varchar,
    ReviewComm varchar
"""
create_table_query = f"CREATE TABLE IF NOT EXISTS {table_name} ({columns})"
cur.execute(create_table_query)
conn.commit()

# Insert the CSV data into the table
for index, row in data.iterrows():
    insert_query = f"INSERT INTO {table_name} ({', '.join(data.columns)}) VALUES ({', '.join(['%s'] * len(data.columns))})"
    cur.execute(insert_query, tuple(row))
conn.commit()

# Close the connection
cur.close()
conn.close()
